### Table of Contents 目录

* [安装数据及建立环境](#chapter1)
* [创建函数](#chapter2)
    * [根据姓+字+朝代（optional）查询人物](#section_2_1)
    * [根据字+朝代（optional）查询人物](#section_2_2)
    * [根据姓+官职（含俗称）+朝代（optional）查询人物](#section_2_3)
* [调用函数](#chapter3)
    * [根据姓+字查询人物](#section_3_1)
    * [根据姓+字+朝代查询人物](#section_3_2)
    * [根据姓+官职（含俗称）查询人物](#section_3_3)
    * [根据姓+官职（含俗称）+朝代查询人物](#section_3_4)

        

<a name="chapter1"></a>
## 安装数据及建立环境 



In [1]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

fatal: destination path 'cbdb_sqlite' already exists and is not an empty directory.


In [2]:
# Explode cbdb_20201110.7z
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('./cbdb_sqlite/latest.7z').extractall(".")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Create connection to database
import sqlite3
global CONN
CONN = sqlite3.connect("./latest.db")

In [4]:
!pip install opencc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<a name="chapter2"></a>
## 创造函数 


<a name="section_2_1"></a>
### 根据姓+字+朝代查询人物（optional）

In [5]:
# Function to search for a person based on surname and alternative name (e.g. 字)
# Dynasty is optional. If not passed in, all results will be returned.
import opencc
s2t_converter = opencc.OpenCC('s2t.json')
def get_person_by_surname_and_altname(surname, altname, dynasty=None):
    surname = s2t_converter.convert(surname)
    altname = s2t_converter.convert(altname)
    if dynasty:
      dynasty = s2t_converter.convert(dynasty)

    cur = CONN.cursor()
    if dynasty:
      sql_statement = "select * from (select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname + "' and BIOG_MAIN.c_surname_chn= '" + surname +"') tmp_db LEFT JOIN DYNASTIES on tmp_db.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='" + dynasty + "';"
    else:
      sql_statement = "select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname + "' and BIOG_MAIN.c_surname_chn='" + surname + "';"
    print(sql_statement)
    cur.execute(sql_statement)
    rows = cur.fetchall()
    count = 0
    for row in rows:
      count += 1
      print("Record"+str(count)+": ")
      print(row)

<a name="section_2_2"></a>
### 根据字+朝代查询人物（optional）

In [6]:
# Function to search for a person based on alternative name (e.g. 字, 号)
# Dynasty is optional. If not passed in, all results will be returned.
import opencc
s2t_converter = opencc.OpenCC('s2t.json')
def get_person_by_altname(altname, dynasty=None):
    altname = s2t_converter.convert(altname)
    if dynasty:
      dynasty = s2t_converter.convert(dynasty)

    cur = CONN.cursor()
    if dynasty:
      sql_statement = "select * from (select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname  +"') tmp_db LEFT JOIN DYNASTIES on tmp_db.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='" + dynasty + "';"
    else:
      sql_statement = "select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname  + "';"
    print(sql_statement)
    cur.execute(sql_statement)
    rows = cur.fetchall()
    count = 0
    for row in rows:
      count += 1
      print("Record"+str(count)+": ")
      print(row)

<a name="section_2_3"></a>
### 根据姓+官职+朝代（optional）查询人物

In [7]:
# Function to search for a person based on surname and office.
# Dynasty is optional. If not passed in, all results will be returned.
import opencc
s2t_converter = opencc.OpenCC('s2t.json')
def get_person_by_surname_and_office(surname, office, dynasty=None):
    surname = s2t_converter.convert(surname)
    office = s2t_converter.convert(office)
    if dynasty:
      dynasty = s2t_converter.convert(dynasty)
    cur = CONN.cursor()
    if dynasty:
      sql_statement = "select * from (select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%" + office + "%' or OFFICE_CODES.c_office_chn like '%" + office + "%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '" + surname +"') tmp_db2 LEFT JOIN DYNASTIES on tmp_db2.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='" + dynasty +"';"  
    else:
      sql_statement = "select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%" + office + "%' or OFFICE_CODES.c_office_chn like '%" + office + "%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '" + surname + "';"
    print(sql_statement)
    cur.execute(sql_statement)
    rows = cur.fetchall()
    count = 0
    for row in rows:
      count += 1
      print("Record"+str(count)+": ")
      print(row)

<a name="chapter3"></a>
## 调用函数 

<a name="section_3_1"></a>
### 根据姓+字查询人物

In [8]:
# Example: Get person by surname + altname
surname = "苏"
zi = "干宝"
get_person_by_surname_and_altname(surname, zi)

select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='幹寶' and BIOG_MAIN.c_surname_chn='蘇';


<a name="section_3_1"></a>
### 根据姓+字+朝代查询人物 

In [9]:
# Example: Get person by surname + altname + dynasty
surname = "柯"
zi = "绍庭"
dynasty = "清"
get_person_by_surname_and_altname(surname, zi, dynasty)

select * from (select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='紹庭' and BIOG_MAIN.c_surname_chn= '柯') tmp_db LEFT JOIN DYNASTIES on tmp_db.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='清';


<a name="section_3_3"></a>
### 根据根据姓+官职（含俗称）查询人物

In [10]:
# Example: Get person by surname + office
surname = "张"
office = "平章"
get_person_by_surname_and_office(surname, office)

select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%平章%' or OFFICE_CODES.c_office_chn like '%平章%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '張';
Record1: 
(183, '中書門下平章事;門下平章;同平章事;平章事', '同中書門下平章事', 182, 183, 'Zhang Qixian', '張齊賢', 943, '01', None, 0, 100409, 1, 0, 0, None, 943, 0, None, None, 1014, 515, 7, None, 72, None, None, 0, None, None, None, 0, None, None, 'Zhang', '張', 'Qixian', '齊賢', 15, 0, "Zhang(1) Qixian [183] His family moved to Luoyang when he was 3 because of disorder during the Five Dynasties. Zonghui's [305] and Zongjian's [3167] father, Zigao's [7171] grandfather. Father-in-law of Wang(2) Juzheng's [8145] brother, Jushan [3945]. Jushan's son-in-law was Sheng(1) Du's [8109] elder cousin Jing's [3739] son, Zunfu [374

<a name="section_3_4"></a>
### 根据姓+官职（含俗称）+朝代查询人物

In [11]:
# Example: Get person by surname + office + dynasty
surname = "王"
office = "相公"
dynasty = "宋"
get_person_by_surname_and_office(surname, office, dynasty)

select * from (select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%相公%' or OFFICE_CODES.c_office_chn like '%相公%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '王') tmp_db2 LEFT JOIN DYNASTIES on tmp_db2.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='宋';
Record1: 
(1762, '參政;知政事;參;大參;參大政;參機政;參鬷嘏;首參;相公;參預政事;參預', '參知政事', 1913, 1762, 'Wang Anshi', '王安石', 1021, '01', None, 0, 100513, 1, 0, 0, None, 1021, 516, 5, None, 1086, 530, 1, None, 66, None, None, 0, None, None, None, 0, None, None, 'Wang', '王', 'Anshi', '安石', 15, 20, "Wang(2) Anshi [1762] Yi(3)'s [7082] son, Guan(1)'s [1841] grandnephew, Anguo's [7076], Anli's [1760], and Anshang's [1761] brother, Fang's [1803] uncle, Jue(1)'s [1796] great grandfather, Zhu(1) Mingzhi's [526] and S

In [12]:
# Example: Get person by surname + office + dynasty
surname = "刘"
office = "运使"
dynasty = "宋"
get_person_by_surname_and_office(surname, office, dynasty)

select * from (select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%運使%' or OFFICE_CODES.c_office_chn like '%運使%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '劉') tmp_db2 LEFT JOIN DYNASTIES on tmp_db2.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='宋';
Record1: 
(1181, '發運;部使者', '發運使', 1369, 1181, 'Liu Chengyan', '劉承顏', None, '', None, 0, 0, 1, 0, 0, None, None, 0, None, None, None, 0, None, None, None, None, None, 0, None, None, None, 0, None, None, 'Liu', '劉', 'Chengyan', '承顏', 15, 0, 'Liu(2) Chengyan [1181] XCB, 107.6b, 114.17b.', 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, 'TTS', '20070312', None, None, 0, 15, 'Song', '宋', 960, 1279, 66)
Record2: 
(1182, '計使', '轉運使', 1370, 1182, 'Liu Chu', '劉楚'